In [1]:
import pandas as pd
from ch2.data import *
from ch2.data.power import *

s = session('-v 5')
df = activity_statistics(s, DISTANCE, ELEVATION, SPEED, CADENCE, LATITUDE, LONGITUDE, HEART_RATE,
                         local_time='2018-03-04 07:16:33', activity_group_name='Bike', with_timespan=True)
print(df.describe())
df = linear_resample(df)
df = add_differentials(df)
print(df.describe())

    INFO: Using database at /home/andrew/.ch2/database.sqlr
    INFO: Using Activity Journal ActivityJournal Bike 2018-03-04 10:16:33 to 2018-03-04 16:34:51


          Latitude    Longitude        Speed       Distance      Cadence  \
count  2885.000000  2885.000000  2886.000000    2933.000000  2903.000000   
mean    -33.473723   -70.806271     5.694779   52424.331323    61.558043   
std       0.034321     0.108272     2.914509   29499.017849    45.063903   
min     -33.549516   -70.949936     0.000000       0.000000     0.000000   
25%     -33.501597   -70.922318     3.154000   29534.110000     0.000000   
50%     -33.460546   -70.826145     6.196000   52038.000000    87.000000   
75%     -33.448303   -70.706109     7.539000   79518.980000   100.000000   
max     -33.420239   -70.607797    13.175000  102708.060000   159.000000   

        Heart Rate    Elevation   Timespan ID  
count  2933.000000  2784.000000   2832.000000  
mean    114.568360   580.671118  19846.668079  
std      13.942284   149.218650     29.986904  
min      65.000000   401.366177  19787.000000  
25%     105.000000   479.115404  19819.000000  
50%     115.000000   518.72

In [2]:
model = PowerModel(wind_speed=100)
print(model)
model = fit_power(df, model, 'slope', 'delay', 'wind_speed', 'wind_heading', tol=0.01)
print(model)
df = evaluate(df, model, quiet=False)
df = add_modeled_hr(df, model.window, model.slope, model.delay)
print(chisq(DETRENDED_HEART_RATE, PREDICTED_HEART_RATE, df))

   DEBUG: Fit power: varying ('slope', 'delay', 'wind_speed', 'wind_heading')


PowerModel(cda=0.5, crr=0, slope=200, window=3600, delay=13.5, m=70, wind_speed=100, wind_heading=0)


   DEBUG: Initial fit LinregressResult(slope=0.00019014270940271213, intercept=117.45955524781768, rvalue=0.10550325650982802, pvalue=6.687917669319006e-08, stderr=3.5113922337071174e-05)
   DEBUG: Fit power: initial model PowerModel(cda=0.5, crr=0, slope=0.00019014270940271213, window=3600, delay=7.0, m=70, wind_speed=100, wind_heading=0)
   DEBUG:  final_simplex: (array([[ 0.01384146, 10.28415549, -0.17998863, -0.03882135],
       [ 0.01384322, 10.28440969, -0.17999326, -0.03882617],
       [ 0.01384115, 10.28391757, -0.18000738, -0.0388202 ],
       [ 0.01383676, 10.28630958, -0.1799298 , -0.0388127 ],
       [ 0.01383853, 10.28517137, -0.17997535, -0.03881547]]), array([181301.91935832, 181301.92277758, 181301.92393721, 181301.92754529,
       181301.92933979]))
           fun: 181301.9193583191
       message: 'Optimization terminated successfully.'
          nfev: 601
           nit: 354
        status: 0
       success: True
             x: array([ 0.01384146, 10.28415549, -0.17

PowerModel(cda=0.5, crr=0, slope=0.013841464977651423, window=3600, delay=11.28415548943445, m=70, wind_speed=0.17998863033241927, wind_heading=179.96117864527304)
181301.91935831908


In [3]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()
predicted = df.copy()
predicted[DETRENDED_HEART_RATE] = predicted[PREDICTED_HEART_RATE]
pl = comparison_line_plot(800, 300, DISTANCE, DETRENDED_HEART_RATE, source=df, other=predicted)
show(pl)
pl = multi_dot_plot(800, 300, DETRENDED_HEART_RATE, [PREDICTED_HEART_RATE], df, ['black'])
show(pl)

Loading BokehJS ...